In [8]:
import lightgbm as lgb
import numpy as np
import os
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, matthews_corrcoef

In [4]:


# Path to the folder containing your .csv and .57pfeatures files
folder_path = '/home/tzejet/drb_jet/output_features'

# Path to the full_ids.txt file
full_ids_path = '/home/tzejet/drb_jet/full_ids.txt'

# Read the list of file prefixes from full_ids.txt
with open(full_ids_path, 'r') as f:
    file_prefixes = [line.strip() for line in f.readlines()]

# Loop through each prefix from the full_ids.txt file
for prefix in file_prefixes:
    # Construct the expected file names for .csv and .57pfeatures
    csv_file_name = f"{prefix}.csv"
    pfeatures_file_name = f"{prefix}.57pfeatures"
    
    # Check if both the .csv and .57pfeatures files exist in the folder
    csv_file_path = os.path.join(folder_path, csv_file_name)
    pfeatures_file_path = os.path.join(folder_path, pfeatures_file_name)
    
    if os.path.exists(csv_file_path) and os.path.exists(pfeatures_file_path):
        # Load the .csv file
        csv_file = pd.read_csv(csv_file_path, header=0)
        
        # Drop the first four columns from the .csv file
        csv_file = csv_file.iloc[:, 4:]
        
        # Load the .57pfeatures file (assuming it's space-separated and text-based)
        with open(pfeatures_file_path, 'r') as pfeatures_file:
            pfeatures_data = pfeatures_file.readlines()[1: ]  # Remove the header
        
        # Remove any '+' characters and convert to numerical format
        pfeatures_array = np.array([list(map(float, line.replace('+', '').split()[1:])) for line in pfeatures_data])
        
        # Convert the .csv data to a NumPy array
        csv_array = csv_file.to_numpy()
        
        # Concatenate the .57pfeatures array and the .csv array row-wise (column-wise merge)
        combined_array = np.hstack((pfeatures_array, csv_array))
        
        # Save the combined array to a new file (or any desired format)
        output_file_name = f"{prefix}_combined.csv"
        output_file_path = os.path.join(folder_path, output_file_name)
        np.savetxt(output_file_path, combined_array, delimiter=',', fmt='%.6f')
        
        print(f"Processed and saved combined file: {output_file_name}")
    else:
        print(f"Missing .csv or .57pfeatures file for prefix: {prefix}")

print("Processing completed!")


Processed and saved combined file: 1A340_combined.csv
Processed and saved combined file: 1A350_combined.csv
Processed and saved combined file: 1A3Q1_combined.csv
Processed and saved combined file: 1A9N1_combined.csv
Processed and saved combined file: 1A9N2_combined.csv
Processed and saved combined file: 1AIS0_combined.csv
Processed and saved combined file: 1AKH1_combined.csv
Processed and saved combined file: 1AM91_combined.csv
Processed and saved combined file: 1AU71_combined.csv
Processed and saved combined file: 1AWC0_combined.csv
Processed and saved combined file: 1B2M1_combined.csv
Processed and saved combined file: 1B3T0_combined.csv
Processed and saved combined file: 1B720_combined.csv
Processed and saved combined file: 1BDT2_combined.csv
Processed and saved combined file: 1BG10_combined.csv
Processed and saved combined file: 1BL00_combined.csv
Processed and saved combined file: 1BRN0_combined.csv
Processed and saved combined file: 1BY41_combined.csv
Processed and saved combined

In [10]:
import os
import pandas as pd
import numpy as np

# Paths to your folder containing combined CSV files and ID files
folder_path = 'output_features'  # Folder with the combined CSV files
train_ids_path = 'train_ids.txt'  # File containing train ID prefixes
test_ids_path = 'test_ids.txt'    # File containing test ID prefixes

# Paths to the DRNA_TRAIN and DRNA_TEST files that contain IDs and labels
train_labels_path = 'DRNA_TRAIN.csv'
test_labels_path = 'DRNA_TEST.csv'

# Step 1: Read the list of prefixes for training and testing
with open(train_ids_path, 'r') as f:
    train_ids = [line.strip() for line in f.readlines()]

with open(test_ids_path, 'r') as f:
    test_ids = [line.strip() for line in f.readlines()]

# Step 2: Load the sequence IDs and labels from the DRNA_TRAIN and DRNA_TEST files
train_labels_df = pd.read_csv(train_labels_path)  # Load as DataFrame
test_labels_df = pd.read_csv(test_labels_path)  # Load as DataFrame

# Step 3: Create helper functions to get labels based on the prefix
def get_label_for_id(prefix, df):
    """Retrieve the label (sequence of 0s and 1s) for a given ID prefix from the DataFrame."""
    row = df[df['id'] == prefix]  # Assuming the CSV has an 'id' column with prefixes
    if row.empty:
        return None  # If no match is found, return None
    return row['dna_label'].values[0]  # Assuming there's a 'label' column

# Initialize lists to hold CSV data rows for train and test sets
train_rows = []
test_rows = []

# Step 4: Loop through all CSV files in the folder and process them
for file_name in os.listdir(folder_path):
    if file_name.endswith('_combined.csv'):
        # Get the ID prefix (assuming the format is ID_combined.csv)
        prefix = file_name.split('_combined.csv')[0]

        # Load the CSV file as a 2D list (not a NumPy array)
        file_path = os.path.join(folder_path, file_name)
        data_array = pd.read_csv(file_path, header=None).values  # Load as a 2D array

        # Step 5: Get the corresponding label for the ID
        if prefix in train_ids:
            label = get_label_for_id(prefix, train_labels_df)
            if label is not None:
                train_rows.append([prefix, label] + data_array.flatten().tolist())
        elif prefix in test_ids:
            label = get_label_for_id(prefix, test_labels_df)
            if label is not None:
                test_rows.append([prefix, label] + data_array.flatten().tolist())
        else:
            print(f"Warning: Prefix {prefix} not found in train or test sets!")

# Step 6: Save train and test rows into separate CSV files
train_output_path = 'train_data_with_labels.csv'
test_output_path = 'test_data_with_labels.csv'

# Save train data to CSV
train_df = pd.DataFrame(train_rows)
train_df.to_csv(train_output_path, index=False, header=False)

# Save test data to CSV
test_df = pd.DataFrame(test_rows)
test_df.to_csv(test_output_path, index=False, header=False)

print(f"Processed and saved train data to {train_output_path}")
print(f"Processed and saved test data to {test_output_path}")


Processed and saved train data to train_data_with_labels.csv
Processed and saved test data to test_data_with_labels.csv


In [3]:
import numpy as np
import pandas as pd
from pathlib import Path

def read_ids(file_path):
    with open(file_path, 'r') as f:
        return [line.strip() for line in f.readlines()]

def create_windows(data, window_size=9):
    # print(len(data))
    windows = []
    for i in range(0, len(data) - window_size + 1):
        window = data[i:i + window_size].to_numpy()
        windows.append(window)
    return np.array(windows)

def process_data(file_ids, label_csv, data_folder, window_size=9):
    data_list = []
    labels_list = []

    # Load the labels file
    labels_df = pd.read_csv(label_csv)

    # Process each file in the file_ids list
    for file_id in file_ids:
        file_path = data_folder / f"{file_id}_combined.csv"
        
        # Read the CSV file
        data = pd.read_csv(file_path, header=None)
        
        # Create windows from the data
        windows = create_windows(data, window_size=window_size)
        
        # Append the windows to data_list
        data_list.append(windows)

        # Get corresponding labels
        label_row = labels_df.loc[labels_df['id'] == file_id]
        labels = label_row['dna_label'].values[0]
        labels = [int(x) for x in labels]  # Convert to integer list
        
        # Skip the first and last 4 labels based on the window size
        labels = labels[4:len(labels) - 4]
        labels_list.append(labels)

    return np.array(data_list), np.array(labels_list)

# Paths and constants
train_ids_path = '/home/tzejet/drb_jet/train_ids.txt'
test_ids_path = '/home/tzejet/drb_jet/test_ids.txt'
train_labels_path = 'DRNA_TRAIN.csv'
test_labels_path = 'DRNA_TEST.csv'
data_folder = Path('/home/tzejet/drb_jet/output_features')  # Change this to the correct folder path

# Step 1: Read train and test file IDs
train_ids = read_ids(train_ids_path)
test_ids = read_ids(test_ids_path)

# Step 2: Process the train data and labels
train_data, train_label = process_data(train_ids, train_labels_path, data_folder)

# Step 3: Process the test data and labels
test_data, test_label = process_data(test_ids, test_labels_path, data_folder)

# Now train_data, train_label, test_data, test_label are ready for further use\
    

/tmp/ipykernel_1256826/1782520410.py:46: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(data_list), np.array(labels_list)
/tmp/ipykernel_1256826/1782520410.py:46: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(data_list), np.array(labels_list)


In [4]:
# Initialize an empty list to store the flattened arrays
flattened_data = []

# Loop through each element in train_data
for sequence in train_data:
    # Reshape each (n, 9, 76) array to (n, 9 * 76)
    n, _, _ = sequence.shape
    flattened_sequence = sequence.reshape(n, 9 * 76)
    
    # Append to the list
    flattened_data.append(flattened_sequence)

# Concatenate all flattened sequences into a single array
# This results in a shape of (x, 9 * 76), where x is the total sum of n's
flattened_train_data = np.vstack(flattened_data)

# Now flattened_train_data is ready, and its shape will be (x, 9 * 76)
print(flattened_train_data.shape)

# Initialize an empty list to store the flattened arrays
flattened_data = []

# Loop through each element in train_data
for sequence in test_data:
    # Reshape each (n, 9, 76) array to (n, 9 * 76)
    n, _, _ = sequence.shape
    flattened_sequence = sequence.reshape(n, 9 * 76)
    
    # Append to the list
    flattened_data.append(flattened_sequence)

# Concatenate all flattened sequences into a single array
# This results in a shape of (x, 9 * 76), where x is the total sum of n's
flattened_test_data = np.vstack(flattened_data)

# Now flattened_train_data is ready, and its shape will be (x, 9 * 76)
print(flattened_test_data.shape)

flattened_train_label = np.concatenate(train_label).ravel()
print(flattened_train_label.shape)

flattened_test_label = np.concatenate(test_label).ravel()
print(flattened_test_label.shape)

(105004, 684)
(19644, 684)
(105004,)
(19644,)


In [5]:
# Remove said rows if the corresponding labels at the index are '2'

# Find the indices of rows with label '2'
indices_to_remove = np.where(flattened_train_label == 2)[0]

# Remove the rows with label '2' from the flattened_train_data and flattened_train_label arrays
filtered_train_data = np.delete(flattened_train_data, indices_to_remove, axis=0)

filtered_train_label = np.delete(flattened_train_label, indices_to_remove)

indices_to_remove = np.where(flattened_test_label == 2)[0]

# Remove the rows with label '2' from the flattened_train_data and flattened_train_label arrays
filtered_test_data = np.delete(flattened_test_data, indices_to_remove, axis=0)

filtered_test_label = np.delete(flattened_test_label, indices_to_remove)

print(filtered_train_data.shape)
print(filtered_train_label.shape)
print(filtered_test_data.shape)
print(filtered_test_label.shape)


(100658, 684)
(100658,)
(18564, 684)
(18564,)


In [6]:
def shuffle(random_state=None):
    positive_indices = np.where(filtered_train_label == 1)[0]
    negative_indices = np.where(filtered_train_label == 0)[0]

    # Determine the number of samples to subsample based on the smaller class
    n_samples = min(positive_indices.shape[0], negative_indices.shape[0]) 

    # Randomly select n_samples from both positive and negative indices
    positive_subsample_indices = np.random.choice(positive_indices, n_samples, replace=False)
    negative_subsample_indices = np.random.choice(negative_indices, n_samples, replace=False)

    # Concatenate the subsampled indices and then use them to create subsampled filtered_train_data and filtered_train_label
    subsample_indices = np.concatenate([positive_subsample_indices, negative_subsample_indices])
    filtered_train_data_subsampled = filtered_train_data[subsample_indices]
    filtered_train_label_subsampled = filtered_train_label[subsample_indices]
    print(filtered_train_data_subsampled.shape, filtered_train_label_subsampled.shape)

    shuffle_indices = np.random.permutation(len(filtered_train_data_subsampled))
    filtered_train_data_subsampled = filtered_train_data_subsampled[shuffle_indices]
    filtered_train_label_subsampled = filtered_train_label_subsampled[shuffle_indices]
    
    return filtered_train_data_subsampled, filtered_train_label_subsampled

filtered_train_data_subsampled, filtered_train_label_subsampled = shuffle()

(15244, 684) (15244,)


In [9]:
train_dataset = lgb.Dataset(filtered_train_data_subsampled, label=filtered_train_label_subsampled)

params = {
    'objective': 'binary',  # Binary classification
    'metric': 'binary_logloss',  # Metric to minimize
    'boosting_type': 'gbdt',  # Gradient Boosting Decision Trees
    'num_leaves': 50,  # Control model complexity, default is 31 (can be tuned)
    'learning_rate': 0.05,  # Controls how fast the model learns (can reduce to 0.01 for better performance)
}

num_round = 100

model = lgb.train(params, train_dataset, num_round)





[LightGBM] [Info] Number of positive: 7622, number of negative: 7622
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.096896 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 112728
[LightGBM] [Info] Number of data points in the train set: 15244, number of used features: 684
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000


In [11]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, matthews_corrcoef

def highlight_max(s):
    is_max = s == s.max()
    return ['background-color: purple' if v else '' for v in is_max]

# Define the range of threshold values
thresholds = np.arange(0.0, 1.00, 0.02)
y_pred_proba = model.predict(filtered_test_data)

# Initialize lists to store the results
results = {
    'Threshold': [],
    'MCC': [],
    'Sensitivity (Recall)': [],
    'Specificity': []
}

# Calculate metrics for each threshold
for threshold in thresholds:
    # Predict the class labels based on the current threshold
    y_pred = (y_pred_proba >= threshold).astype(int)
    
    # Calculate MCC
    mcc = matthews_corrcoef(filtered_test_label, y_pred)
    
    # Calculate Sensitivity (Recall)
    sensitivity = recall_score(filtered_test_label, y_pred)
    
    # Calculate Specificity
    tn, fp, fn, tp = confusion_matrix(filtered_test_label, y_pred).ravel()
    specificity = tn / (tn + fp)
    
    # Store the results
    results['Threshold'].append(threshold)
    results['MCC'].append(mcc)
    results['Sensitivity (Recall)'].append(sensitivity)
    results['Specificity'].append(specificity)

# Create a DataFrame from the results
results_df = pd.DataFrame(results)
results_df = results_df.round(2)
# Apply the highlight function only to the 'MCC' column
styled_results_df = results_df.style.format("{:.2f}").apply(highlight_max, subset=['MCC'], axis=0)

# Display the styled DataFrame
styled_results_df


# Optionally, save the DataFrame to a CSV file
# results_df.to_csv('threshold_metrics.csv', index=False)

,Threshold,MCC,Sensitivity (Recall),Specificity
0,0.00,0.00,1.00,0.00
1,0.02,0.00,1.00,0.00
2,0.04,0.02,1.00,0.01
3,0.06,0.05,1.00,0.04
4,0.08,0.06,1.00,0.08
5,0.10,0.08,0.99,0.12
6,0.12,0.09,0.98,0.16
7,0.14,0.10,0.98,0.20
8,0.16,0.11,0.97,0.24
9,0.18,0.12,0.96,0.27
